# Reading Bhavcopy from the given bhavcopy.txt

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import pandas_datareader.data as web
import datetime as dt
from datetime import date, time
import os 
import struct


f = open('C:\\Users\\a\\Desktop\\Mohit Project\\faomktstats20180516.txt', 'rb') #Enter the file path
b = bytearray(f.read())
f.close()
 

i = 0
instrument = list()
oi = list()
oi_change = list()
close_price = []


while True:
    try:
        data = struct.unpack('<6s10s11s10s2s1s10s10s10s10s10s10s10s12s25s10s10s', b[i:i+167])  # Data structure
        strike = str(float(data[3].strip()))
        expiry_date = dt.datetime.strptime(bytes(data[2].strip()).decode('ascii'), '%d-%m-%Y')
        expiry_date = dt.datetime.strftime(expiry_date, '%d-%b-%Y')
        instrument.append(bytes(data[1].strip()).decode('ascii') + '_' + expiry_date + '_' + strike + '_' + bytes(data[4]).decode('ascii'))
        oi.append(bytes(data[15].strip()).decode('ascii'))
        oi_change.append(bytes(data[16].strip()).decode('ascii'))
        close_price.append(float(data[9].strip()))
        i += 169
    except:
        break

rows= zip(instrument, oi, close_price)

import csv

with open('C:\\Users\\a\\Desktop\\Mohit Project\\bhavcopy_from_text.csv', "w", newline = "") as f: #Enter the bhavcopy file path here
    bhavcopy_directory = ('C:\\Users\\a\\Desktop\\Mohit Project\\')
    writer = csv.writer(f)
    writer.writerow(("INSTRUMENT","OI","CLOSE"))
    for row in rows:
        writer.writerow(row)
f.close()

 

In [2]:
os.chdir(bhavcopy_directory)
bhavcopy_from_text = pd.read_csv('bhavcopy_from_text.csv')

In [3]:
bhavcopy_from_text.head()

,INSTRUMENT,OI,CLOSE
0,NIFTYCPSE_17-May-2018_2225.0_CE,0,205.85
1,NIFTYCPSE_17-May-2018_2675.0_PE,0,157.90
2,NIFTYCPSE_17-May-2018_2250.0_CE,0,208.30
3,NIFTYCPSE_17-May-2018_2250.0_PE,0,21.25
4,NIFTYCPSE_17-May-2018_2275.0_CE,0,188.95


In [4]:
bhavcopy_from_text['EXPIRY_DT'] = bhavcopy_from_text['INSTRUMENT'].apply(lambda x: x.split("_")[1])
bhavcopy_from_text.head()

,INSTRUMENT,OI,CLOSE,EXPIRY_DT
0,NIFTYCPSE_17-May-2018_2225.0_CE,0,205.85,17-May-2018
1,NIFTYCPSE_17-May-2018_2675.0_PE,0,157.90,17-May-2018
2,NIFTYCPSE_17-May-2018_2250.0_CE,0,208.30,17-May-2018
3,NIFTYCPSE_17-May-2018_2250.0_PE,0,21.25,17-May-2018
4,NIFTYCPSE_17-May-2018_2275.0_CE,0,188.95,17-May-2018


In [5]:
pd.options.mode.chained_assignment = None  #Turning off the 'SettingWithCopyWarning'

# Reading the security file (Scrip Codes , KEY) 

In [6]:
os.chdir('C:\\Users\\a\\Desktop\\Mohit Project')

In [7]:
import datetime
import os 
import struct

In [8]:
f = open('C:\\Users\\a\\Desktop\\Mohit Project\\Securities.dat' , 'rb') #Enter the file path
b = bytearray(f.read())
f.close()

# Make lists to store values
scrip_code = list()
instrument = list()
short_code = list()
opt_type = list()
strikes = list()
ttl_time = list()
i = 0
# Loop through values
seconds_toadd = 315599400
while True:
    try:
        data = struct.unpack('<hlhl6s10s2s2sllllll25sllllll25s1s1s', b[i:i+132])  # Data structure
    
        if data[7] != 'XX' and data[5].strip() != '':
            
            
            scrip_code.append(data[3])
            start_date = dt.datetime.strptime('01-01-1980', '%d-%m-%Y')         
            ttl_time.append(int(data[8] + seconds_toadd))
            expiry_date = dt.datetime.strftime(start_date + dt.timedelta(seconds=int(data[8])), '%d-%b-%Y')
            strike = str(float(data[9]) / 100)
            instrument.append(bytes(data[5].strip()).decode('ascii') + '_' + expiry_date + '_' + strike + '_' + bytes(data[7]).decode('ascii'))
            short_code.append(bytes(data[5].strip()).decode('ascii') + '_' + expiry_date)
            strikes.append(strike)
            opt_type.append(bytes(data[7].strip()).decode('ascii'))
            
        i += 132
    except:
        
        break

rows= zip(scrip_code, ttl_time, instrument, short_code, strikes, opt_type)
import csv

with open('C:\\Users\\a\\Desktop\\Mohit Project\\SECDATA3.csv', "w",newline='') as f: #enter the security file path 
    writer = csv.writer(f)
    writer.writerow(("SCRIP_CODE","TTL_TIME","INSTRUMENT","SHORT_CODE","STRIKES","OPT_TYP"))
    for row in rows:
        writer.writerow(row),
f.close()



In [9]:
security_file = pd.read_csv('SECDATA3.csv')
security_file.head()

,SCRIP_CODE,TTL_TIME,INSTRUMENT,SHORT_CODE,STRIKES,OPT_TYP
0,35000,315599400,_01-Jan-1980_0.0_,_01-Jan-1980,0.0,NaN
1,35001,315599400,_01-Jan-1980_0.0_,_01-Jan-1980,0.0,NaN
2,35002,1609491600,NIFTY_31-Dec-2020_10400.0_CE,NIFTY_31-Dec-2020,10400.0,CE
3,35003,1609491600,NIFTY_31-Dec-2020_10400.0_PE,NIFTY_31-Dec-2020,10400.0,PE
4,35004,315599400,_01-Jan-1980_0.0_,_01-Jan-1980,0.0,NaN


# Processing the security file

In [10]:
def process_security_file(file):
    df = pd.read_csv(file)  #read from the running file called 'SECDATA.csv' , stored locally 
    df.drop(['SHORT_CODE','STRIKES','OPT_TYP','TTL_TIME'],axis = 1,  inplace = True) #drop the redundant columns
    df = df[df['INSTRUMENT'].str.contains('_01-Jan-80_0.0_') == False]
    df.rename(index = str, columns = {'INSTRUMENT': 'KEY'}, inplace = True)
    
    #df = df.merge('The working file', how = 'outer', on = 'KEY')
    df =df.sort_values(by = 'KEY')
    #df = df[np.isfinite(df['CLOSE_0'])] #dropping the expired dates' rows
    df.to_csv(file, index = False)
    return df



In [11]:
security = process_security_file('SECDATA3.csv')

security.head()

,SCRIP_CODE,KEY
9510,45139,ACC_26-Jul-2018_-0.01_XX
8185,43185,ACC_26-Jul-2018_1000.0_CE
8187,43187,ACC_26-Jul-2018_1000.0_PE
6921,41921,ACC_26-Jul-2018_1020.0_CE
6922,41922,ACC_26-Jul-2018_1020.0_PE


# Reading the current date


In [12]:
import datetime
current_date = datetime.datetime.today().date()
current_date = datetime.datetime.strftime(current_date, format = '%d-%b-%Y')
current_date

'06-Jun-2018'

# Updating the Bhavcopy file with Scrip_codes

In [14]:
bhavcopy_from_text.rename(columns = {'INSTRUMENT':'KEY'}, inplace = True)
bhavcopy_from_text.head()

,KEY,OI,CLOSE,EXPIRY_DT
0,NIFTYCPSE_17-May-2018_2225.0_CE,0,205.85,17-May-2018
1,NIFTYCPSE_17-May-2018_2675.0_PE,0,157.90,17-May-2018
2,NIFTYCPSE_17-May-2018_2250.0_CE,0,208.30,17-May-2018
3,NIFTYCPSE_17-May-2018_2250.0_PE,0,21.25,17-May-2018
4,NIFTYCPSE_17-May-2018_2275.0_CE,0,188.95,17-May-2018


In [15]:
bhavcopy_from_text = pd.merge(bhavcopy_from_text , security , on = 'KEY' , how = 'outer')
#bhavcopy_from_text.drop('SCRIP_CODE', axis = 1,  inplace= True)
bhavcopy_from_text.head()

,KEY,OI,CLOSE,EXPIRY_DT,SCRIP_CODE
0,NIFTYCPSE_17-May-2018_2225.0_CE,0.0,205.85,17-May-2018,NaN
1,NIFTYCPSE_17-May-2018_2675.0_PE,0.0,157.90,17-May-2018,NaN
2,NIFTYCPSE_17-May-2018_2250.0_CE,0.0,208.30,17-May-2018,NaN
3,NIFTYCPSE_17-May-2018_2250.0_PE,0.0,21.25,17-May-2018,NaN
4,NIFTYCPSE_17-May-2018_2275.0_CE,0.0,188.95,17-May-2018,NaN


In [16]:
bhavcopy_from_text.to_csv('bhavcopy_from_text.csv')

# Reading the OI from the .ind file


In [17]:
import csv
import gzip


In [18]:
from scipy import stats

In [19]:
#import shutil 


In [20]:
flist_ind_files = []   #Creating list of .ind files
flist_sec_files = []   #creating list of .sec files

In [22]:
os.chdir('C:\\Users\\a\\Desktop\\Mohit Project\\Data to be read\\DATA\\May172018\\')
flist1 = [file for file in os.listdir('C:\\Users\\a\\Desktop\\Mohit Project\\Data to be read\\DATA\\May172018\\') if file.endswith('.ind.gz')]

for file in flist1:
    flist_ind_files.append(str(file))
    print("File working on: {}".format(file))
    
    f = gzip.open('C:\\Users\\a\\Desktop\\Mohit Project\\Data to be read\\DATA\\May172018\\' + str(file) , 'rb') #Enter the file path
    b = bytearray(f.read())
    f.close()
    #shutil.move("C:\\Users\\a\\Desktop\\Mohit Project\\Data to be read\\DATA\\May172018\\"+str(file), "C:\\Users\\a\\Desktop\\Mohit Project\\Data to be read\\DATA\\May172018\\Data read")
    #print("Moving file:{}".format(str(file)))
    
    scripCodes = list()
    oi = list()
    instrument = list()
    
   
    c5 = list()
    c6 = list()
    expiry_date = list()
    c8 = list()
    c9 = list()
    c10 = list()
    c11 = list()
    c13 = list()
    i = 0
    data_time = 0  
      
    while True:
            
            try:
                data = struct.unpack('<hlhcl6s10s11s10s2s10schc', b[i:i+66])
                data_time = max(data[1], data_time)  # For getting time of data as max within file
                oi.append(float(data[10].strip()))
                scripCodes.append(data[4])
                #c5.append(bytes(data[5].strip()).decode('ascii'))
                #c6.append(bytes(data[6].strip()).decode('ascii'))
                expiry_date.append(bytes(data[7].strip()).decode('ascii'))
                #c8.append(float(data[8].strip()))
                #c9.append(bytes(data[9].strip()).decode('ascii'))
                
                
                i += 66
                
            except:
                
                "except block run"
                break
    #Checking if the file is zero size 
    if len(oi) == 0:
        print("File {} is empty".format(file))
    print(len(oi))    
    rows= zip(scripCodes, oi,expiry_date)
   
   
            
    with open('C:\\Users\\a\\Desktop\\Mohit Project\\Files read\\'+str(file)+'.csv', "w", newline = "") as f: #Enter the file path
        #print("Writing")
        writer = csv.writer(f)
        writer.writerow(("SCRIP_CODE","OI","EXPIRY_DT"))
        
        for row in rows:
            writer.writerow(row)
            
    f.close()

print(flist_ind_files)    

File working on: 1.ind.gz
File 1.ind.gz is empty
0
File working on: 2.ind.gz
1777
File working on: 3.ind.gz
1875
File working on: 4.ind.gz
1851
File working on: 5.ind.gz
1892
File working on: 6.ind.gz
1800
['1.ind.gz', '1.ind.gz', '2.ind.gz', '3.ind.gz', '4.ind.gz', '5.ind.gz', '6.ind.gz']


# Reading the Close price and scrip codes from the '.sec' file

In [24]:
#Enter the file path in the next line
flist2 = [file for file in os.listdir('C:\\Users\\a\\Desktop\\Mohit Project\\Data to be read\\DATA\\May172018\\') if file.endswith('.sec.gz')]

import  numpy
timestamp = []
flist_bad = []
instrument = []
symbol = []

strike_price = []
option_type = []





for file in flist2:
    i = 0
    data_time = 0
    expiry_date = []
    scripCodes = list()
    close = list()             
    f = gzip.open('C:\\Users\\a\\Desktop\\Mohit Project\\Data to be read\\DATA\\May172018\\' + str(file) , 'rb') #Enter the file path
    try:
        b = bytearray(f.read())
    except:
        flist_bad.append(file)
    f.close()
    
    #shutil.move("C:\\Users\\a\\Desktop\\Mohit Project\\Data to be read\\DATA\\May172018\\"+str(file), "C:\\Users\\a\\Desktop\\Mohit Project\\Data to be read\\DATA\\May172018\\Data read\\.sec files")
    while True:
          
        try:
            
            
            data = struct.unpack('<hlhcl6s10s11s10s2s1s10s12s10s12s10s12s10s1s10s10s10s10s', b[i:i+170]) #Data structure
            
            
            data_time = max(data[1], data_time)  # For getting time of data as max within file
            #timestamp.append(data[1])
            #instrument.append(bytes(data[5]).decode('ascii'))
            #symbol.append(bytes(data[6]).decode('ascii'))
            expiry_date.append(bytes(data[7]).decode('ascii'))
            #strike_price.append(bytes(data[8]).decode('ascii'))
            #option_type.append(bytes(data[9]).decode('ascii'))
            if float(bytes(data[15]).strip()) ==0.00  or 0 or 0.0:
                close.append(numpy.nan)
                #print("Was nan")
                
            else: 
                close.append(float((data[15]).strip()))
            
            scripCodes.append(data[4])
              
            
            i += 225
            
            
        except:
            
               
            break
    #Checking if the file is zero size 
    
    
    if len(close) == 0 :
        
        print("File {} is empty".format(file))
    #print(close)
    rows= zip(scripCodes, close, expiry_date)
    
    import csv

    

    with open('C:\\Users\\a\\Desktop\\Mohit Project\\{}.csv'.format(file) , "w", newline = "") as f: #Enter the file path 
        
        writer = csv.writer(f)
        writer.writerow(("SCRIP_CODE","CLOSE", "EXPIRY_DT"))
        for row in rows:
            writer.writerow(row)
    f.close()



File 1.sec.gz is empty


# Combining '.sec' file and '.ind' file data into the master file


In [25]:
os.getcwd()

'C:\\Users\\a\\Desktop\\Mohit Project\\Data to be read\\DATA\\May172018'

In [26]:
print(flist1)

['1.ind.gz', '2.ind.gz', '3.ind.gz', '4.ind.gz', '5.ind.gz', '6.ind.gz']


In [27]:
print(flist2)

['1.sec.gz', '2.sec.gz', '3.sec.gz', '4.sec.gz', '5.sec.gz', '6.sec.gz']


# Reading the .sec and .ind files in to dataframes

In [28]:
os.chdir('C:\\Users\\a\\Desktop\\Mohit Project') #.sec file directory

In [29]:
flist_ind = [file for file in os.listdir('C:\\Users\\a\\Desktop\\Mohit Project\\Files read') if file.endswith('.ind.gz.csv')]
flist_sec = [file for file in os.listdir('C:\\Users\\a\\Desktop\\Mohit Project') if file.endswith('.sec.gz.csv')]
flist_sec

['1.sec.gz.csv',
 '2.sec.gz.csv',
 '3.sec.gz.csv',
 '4.sec.gz.csv',
 '5.sec.gz.csv',
 '6.sec.gz.csv']

# Creating the list of ind and sec DataFrames

In [30]:
df_list_ind = []
for file in flist_ind:
    
    df= pd.read_csv('C:\\Users\\a\\Desktop\\Mohit Project\\Files Read\\' + file )
    df_list_ind.append(df)
    
len(df_list_ind)

6

In [31]:
import numpy
df_list_sec = []
for file in flist_sec:
    
        df= pd.read_csv('C:\\Users\\a\\Desktop\\Mohit Project\\' +str(file))
        df['CLOSE'].replace(0.00, numpy.nan , inplace = True)
        
        
        df_list_sec.append(df)
        
        
    #df_close_2.head()
    


# Merging the ind and sec DataFrames

In [32]:
#Creating a single list of dataframes, by merging 
df_list = []
for i in range(len(df_list_ind)):
    df = pd.merge(df_list_ind[i],df_list_sec[i], on = 'SCRIP_CODE', how = 'outer')
    
    df.rename(columns = {'EXPIRY_DT_x':'EXPIRY_DT'}, inplace = True)
    df.drop(['EXPIRY_DT_y'], axis = 1, inplace= True)
    
    
    df_list.append(df)
    
    #print(df.head())


# Code to calculate the average price from the preprocessed files.

In [33]:

    

count = 0
for file in flist_ind:
    
    count = count + 1
    if count == 1 :
        if len(df_list_ind[count - 1]['OI']) == 0:
            
            df_list[count - 1] = bhavcopy_from_text
            
           
            df_list[count-1] = df_list[count-1][df_list[count-1]['EXPIRY_DT'] > current_date] #remove the expired dates
            df_list[count-1].rename(columns = {'CLOSE' : 'CLOSE'+ str(count), 'OI' : 'OI'+ str(count)}, inplace = True)
            df_list[count-1]= df_list[count-1][df_list[count-1]['SCRIP_CODE'].isnull() == False] #removing empty scrip code rows
            
        if len(df_list_ind[count - 1]['OI'] )> 0:
            
            df_list[count - 1] = pd.concat([df_list[count - 1],bhavcopy_from_text]).drop_duplicates(subset=['SCRIP_CODE'], keep='last')
            
            
            df_list[count-1] = df_list[count-1][df_list[count-1]['EXPIRY_DT'] > current_date] #remove the expired dates
            df_list[count-1]= df_list[count-1][df_list[count-1]['SCRIP_CODE'].isnull() == False]
            
            df_list[count - 1].rename(columns = {'CLOSE' : 'CLOSE'+ str(count), 'OI' : 'OI'+ str(count)}, inplace = True)
            df_list[count-1]= df_list[count-1][df_list[count-1]['SCRIP_CODE'].isnull() == False] #removing empty scrip code rows
        
        df_list[count - 1 ]['TOTAL_VAL'] = df_list[count-1]['CLOSE'+ str(count)] * df_list[count-1]['OI'+ str(count)]
        master_file = df_list[count-1]
        
        
        '''IF THE EXPIRED INSTRUMENTS ARE TO BE REMOVED UNHASH THE TWO LINES BELOW- STEP 1'''
        
        #master_file['EXPIRY_DT'] = master_file['EXPIRY_DT'].apply(lambda x: pd.to_datetime(x))
        #master_file = master_file[master_file['EXPIRY_DT'] > current_date] #remove the expired dates
        master_file.to_csv(str(count)+ '.csv')
        
    if count >1 :
        print("Count = {}".format(count))
        master_file = pd.merge(master_file, df_list[count - 1], on = 'SCRIP_CODE' , how = 'outer')
        
        
        master_file['EXPIRY_DT_x'].update(master_file['EXPIRY_DT_y'])
        master_file.drop('EXPIRY_DT_y', axis = 1 , inplace = True)
        master_file.rename(columns = {'EXPIRY_DT_x':'EXPIRY_DT'}, inplace = True)
        
        
        '''Unhash the below lines if Expired instruments are to be removed- STEP 2'''
        
        #master_file['EXPIRY_DT'] = pd.to_datetime(master_file.EXPIRY_DT)
        #master_file['EXPIRY_DT'] = master_file['EXPIRY_DT'].apply(lambda x: pd.to_datetime(x))
        #master_file = master_file[master_file['EXPIRY_DT'] > current_date] #remove the expired dates
        
       
        
        
            
        
           
        
        
        master_file['CLOSE_new'] = master_file['CLOSE'+ str(count-1)].copy()
        master_file['CLOSE_new'].update(master_file['CLOSE'])
        master_file['CLOSE'] = master_file['CLOSE_new']
        
        
        
        
       
        
        
        master_file['OI_new'] = master_file['OI'+ str(count-1)]
        master_file['OI_new'].update(master_file['OI'])
        master_file['OI']  = master_file['OI_new']
        
        
        
        master_file.drop(['CLOSE_new','OI_new'], axis = 1 , inplace = True)
  
        
        
        master_file.rename(columns = {'CLOSE' : 'CLOSE'+ str(count), 'OI' : 'OI'+ str(count)}, inplace = True)
        
        master_file['CHG_IN_OI'+str(count)] = master_file['OI'+str(count)] -  master_file['OI'+str(count-1)]
        master_file['NET_CHG_IN_VAL' + str(count)]   = master_file['CHG_IN_OI'+str(count)] * master_file['CLOSE'+ str(count)]  
        if count == 2 :
            master_file['NEW_NET_VAL'+str(count)] = master_file['TOTAL_VAL'] +master_file['NET_CHG_IN_VAL'+str(count)]
        if count > 2: 
            master_file['NEW_NET_VAL'+str(count)] = master_file['NET_CHG_IN_VAL'+str(count)] +master_file['NET_CHG_IN_VAL'+str(count - 1 )]
        master_file['AVG_PRICE' + str(count)] = master_file['NEW_NET_VAL'+str(count)] / master_file['OI'+str(count)]
        master_file.to_csv(str(count)+ '.csv')   
        
        
        
        '''DROPPING REDUNDANT COLUMNS'''
        
        if count ==2:
            
            master_file.drop(['CLOSE'+str(count-1), 'OI' +str(count-1), 'TOTAL_VAL','CHG_IN_OI'+str(count),'NEW_NET_VAL'+str(count) ], axis = 1, inplace = True)
            
        if count > 2:
            master_file.drop(['CLOSE'+str(count-1), 'OI' +str(count-1), 'AVG_PRICE' + str(count-1),'CHG_IN_OI'+ str(count), 'NET_CHG_IN_VAL'+ str(count-1), 'NEW_NET_VAL'+ str(count)],axis = 1 , inplace = True)
        
        
    
    

Count = 2
Count = 3
Count = 4
Count = 5
Count = 6
